In [ ]:
#LSTM modeller devam eden işlemlerde tahmin etmemizi sğalıyor
#We use LSTM models to predict in continous process

In [20]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding
import tensorflow as tf

In [4]:
from google.colab import files
uploaded = files.upload()

Saving TurkishSMSCollection.csv to TurkishSMSCollection.csv


In [5]:
# verimizde noktalı virgül ile ayrılmıştı kolonlar onu hallettik
df = pd.read_csv('TurkishSMSCollection.csv',sep=';', on_bad_lines= 'skip')

In [6]:
df.head(3)

,Message,Group,GroupText
0,125 lira,2,Normal
1,Baskanin aksam toplantısi fenaymis :),2,Normal
2,Bilal yalçnlara ne zaman gidiyoruz?,2,Normal


In [7]:
# burada işte group'daki değer 1 ise 1, 1 değilse de 0 olsun diyoruz
df['labels'] = np.where(df['Group']==1,1,0)

In [8]:
df.head(10)

,Message,Group,GroupText,labels
0,125 lira,2,Normal,0
1,Baskanin aksam toplantısi fenaymis :),2,Normal,0
2,Bilal yalçnlara ne zaman gidiyoruz?,2,Normal,0
3,"BiP ile mesajlarimi aninda, daha eglenceli gon...",1,Spam,1
4,DIGITURKTEN FIRSAT! SiZE OZEL YIL SONUNA KADAR...,1,Spam,1
5,İyi ya dokuz eylül iyidir arş.gör falan kovala ;),2,Normal,0
6,İyiyim teşekkürler oturuyoruz nazarda arkadaşl...,2,Normal,0
7,Kapatamıyorun ayarlara girmem lazım :),2,Normal,0
8,Menüye girsem görünür mü acaba ??,2,Normal,0
9,Önemli değil hocam iyi akşamlar ;),2,Normal,0


In [76]:
# Şimdi bağımlı ve bağımsız değişkenlerimizi seçelim aslında bunu tokenizer ve lstm ile yapicaz
#Tokenizeri oluşturalım, burada kelime sayısını yani maximum önemli bulacağı kelime sayısını 500 olarak ayarlıyoruz
tokenizer = Tokenizer(num_words=500)
#Şimdi de verilerimiz ile tokenizeri eğitiyoruz, dikkat ederseniz text halinde eğitiliyor
tokenizer.fit_on_texts(df['Message'])
#Şimdi de verilerimizi normal string yerine sequence ifadelere çevirelim
sequences = tokenizer.texts_to_sequences(df['Message'])
#Şimdi de yapacağımız şey şu, tümm bu şeçilen şeyleri aynı uzunlukta ayarlayacağız yani sequencelerin uzuklukları sabit 20 olacak, pad bağımsız değişkendir
pad = pad_sequences(sequences,maxlen=20)
y = df['labels'].values

In [77]:
#bağımlı ve bağımsız değişkenlerimizi traintestsplit ile bölelim train ve teste
x_train,x_test,y_train,y_test = train_test_split(pad,y,train_size = 0.75, random_state = 42)

In [96]:
# Şimdi de modelimizi oluşturmaya başlayalım
model = Sequential()
# LSTM haline çevirmek için embedding katmanı ekleyeceğiz, girişimiz num_words yani 500, çıkışımızı kendimiz belirliyoruz
# ---- katmanı
model.add(Embedding(input_dim=500,output_dim=100))
# LSTM'i ekliyoruz, burada dropout ve recurrent_dropout kısımları var, normalde lstm'de biz modeli incelediğimizde forget gate input gate ve output gate vardı, burada belli başlı bilgilerin ileriye dönük olarak tutulup
#tutulmayacağı tanh ve sigmoid fonksiyonlar ile belirleniyordu, bu dropout buna tam olarak benzemese de overfitting'i önlemek için nöronları rastgele seçmemizi sağlar, dropout bunu inputta yaparken recurrent_dropout bunu
#hidden layer'de yapar
# ---- katmanı
model.add(LSTM(64,activation='tanh', recurrent_activation='sigmoid', dropout=0.2,recurrent_dropout=0.2))
# ---- katmanı
model.add(Dense(1,activation='sigmoid'))
# Modelin ayarlanması
model.compile( loss= tf.keras.losses.BinaryCrossentropy(), optimizer='adam', metrics=[tf.keras.metrics.BinaryAccuracy()])

In [97]:
# Modeli eğitelim ama validasyon ile eğitelim yani
model.fit(x_train,y_train,epochs=12,validation_data=(x_test,y_test),batch_size=64)

Epoch 1/12
56/56 ━━━━━━━━━━━━━━━━━━━━ 9s 78ms/step - binary_accuracy: 0.8227 - loss: 0.4145 - val_binary_accuracy: 0.9764 - val_loss: 0.0695
Epoch 2/12
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - binary_accuracy: 0.9847 - loss: 0.0500 - val_binary_accuracy: 0.9815 - val_loss: 0.0516
Epoch 3/12
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - binary_accuracy: 0.9891 - loss: 0.0360 - val_binary_accuracy: 0.9865 - val_loss: 0.0442
Epoch 4/12
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - binary_accuracy: 0.9935 - loss: 0.0210 - val_binary_accuracy: 0.9840 - val_loss: 0.0448
Epoch 5/12
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - binary_accuracy: 0.9949 - loss: 0.0147 - val_binary_accuracy: 0.9882 - val_loss: 0.0421
Epoch 6/12
56/56 ━━━━━━━━━━━━━━━━━━━━ 4s 65ms/step - binary_accuracy: 0.9965 - loss: 0.0100 - val_binary_accuracy: 0.9899 - val_loss: 0.0401
Epoch 7/12
56/56 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - binary_accuracy: 0.9990 - loss: 0.0047 - val_binary_accuracy: 0.9899 - val_loss: 0.0406
Epoch 8/12
56

In [98]:
message = "merhaba dostum nasılsın napıyorsun hiç yazmıyorsun ayıp"
message_to_predict = tokenizer.texts_to_sequences([message])
message_to_predict = pad_sequences(message_to_predict,maxlen=20)
sonuc = model.predict(message_to_predict)
sonuc

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 705ms/step


array([[0.00010938]], dtype=float32)

In [99]:
message_spam = "INDIRIM KAMPANYA GEL AL SERVIS FALAN YAHU COK UCUZ"
message_spam_to_predict = tokenizer.texts_to_sequences([message_spam])
message_spam_to_predict = pad_sequences(message_spam_to_predict,maxlen=20)
sonuc_spam = model.predict(message_spam_to_predict)
sonuc_spam

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


array([[0.99933565]], dtype=float32)

In [100]:
model.summary()

Model: "sequential_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_17 (Embedding)        │ (None, 20, 100)        │        50,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_15 (LSTM)                  │ (None, 64)             │        42,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 276,917 (1.06 MB)

 Trainable params: 92,305 (360.57 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 184,612 (721.14 KB)

In [104]:
#Başarıyı ölçelim test veri seti üzerinden
loss, accuracy = model.evaluate(x_test, y_test)
print("loss",loss)
print("accuracy",accuracy)

38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - binary_accuracy: 0.9889 - loss: 0.0505
loss 0.043172117322683334
accuracy 0.9898989796638489
